![alt text](https://www.business.unsw.edu.au/style%20library/asb/assets/images/logo-unsw.png)


# MARK5826: Matrix factorization 

In this tutorial, we will keep analyzing a dataset which have 100k rating informations contains 943 users and 1682 movies.

## Introduction:
**Matrix factorization**: given a matrix R, we want to find a suitable U (User latent matrix) and V(Item latent matrix) which could reflect the prediction of user and item.

![alt text](https://i.ibb.co/L5Tdfxn/image.png)

**Loss function**: The concept of loss function is simple. it’s a method of evaluating how well your algorithm models your dataset. If your predictions are totally off, your loss function will output a higher number. If they’re pretty good, it’ll output a lower number. 
Machine learning is not a magic, machines just learn by means of a loss function.

### Including files and folders

In [1]:
from google.colab import files

# Upload the dataset movielens_100k
files.upload()

{}

In [2]:
from google.colab import files

# Upload utils.py
files.upload()


{}

## Use the SURPRISE Recommendation Library

### [SURPRISE Library](http://surpriselib.com/)

### Surprise is a Python scikit building and analyzing recommender systems


In [3]:
!pip install scikit-surprise

In [4]:
from utils import *
from surprise.model_selection import cross_validate
from surprise import SVD
import warnings; warnings.simplefilter('ignore')

# Load the data
data, n_users, n_items = spr_loadData('ml-100k.data')

# Select Algorithm SVD and Run It
algo = SVD()
cross_validate(algo, data, measures=['RMSE'], cv=5, verbose=True)

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9307  0.9351  0.9414  0.9413  0.9289  0.9355  0.0052  
Fit time          6.08    6.10    6.04    6.02    6.03    6.05    0.03    
Test time         0.16    0.27    0.16    0.16    0.26    0.20    0.05    


{'fit_time': (6.084584712982178,
  6.097024202346802,
  6.035595417022705,
  6.0150306224823,
  6.034153938293457),
 'test_rmse': array([0.93068572, 0.93505461, 0.94141642, 0.94132203, 0.92888099]),
 'test_time': (0.16487550735473633,
  0.2674708366394043,
  0.16092348098754883,
  0.16403651237487793,
  0.257692813873291)}

## Basic Matrix Factorization Using SVD

#### Rating Prediction By Inner Products
![alt text](https://cdn-images-1.medium.com/max/800/1*WWO2sCXvTdQpAtLEJ5h9Zg.png)

#### Loss Function
![Loss Function](https://cdn-images-1.medium.com/max/800/1*l1oezj1Ze8JbBZttYYVHmA.png)

#### Updating Algorithm
![alt text](https://cdn-images-1.medium.com/max/800/1*MdWVINJBDAOofbgQcyRbCw.png)


In [0]:
from utils import *
import warnings; warnings.simplefilter('ignore')

class MF(object):
    def __init__(self,
                 num_factors = 16,
                 regularizer=0.001,
                 learning_rate=0.001,
                 epochs=100,
                 batch_size=128):

        # Initialize Parameters
        self.num_factors = num_factors # Dimension of the Latent Factor
        self.regs = regularizer # Regularizer Coefficient

        self.lr = learning_rate # Learning Rate
        self.epochs = epochs # How many number of Training Loops
        self.batch_size = batch_size # How many data is fed into the training algorithm in each epoch

    def prepare_data(self, test_size=0.2, datafile='ml-100k.data'):
        # Load Data
        train_matrix, test_matrix, num_user, num_item, uid_min, iid_min = loadData(test_size, datafile)
        self.num_user, self.num_item = num_user, num_item

        # Store the user IDs in a list, the item IDs in a list and the ratings in a list
        train_matrix, test_matrix = train_matrix.tocoo(), test_matrix.tocoo()
        self.train_uid, self.train_iid, self.train_ratings = list(train_matrix.row),list(train_matrix.col),list(train_matrix.data)
        self.test_uid, self.test_iid, self.test_ratings = list(test_matrix.row),list(test_matrix.col),list(test_matrix.data)

        # Calculate the average of all ratings (the mu value in the equation)
        self.mu = np.mean(self.train_ratings)

        # Total number of training data instances
        self.num_training = len(self.train_ratings)

        # Number of batches
        self.num_batch = int(self.num_training / self.batch_size)
        print("Data Preparation Completed.")

    # Build the model for customized SGD algorithm
    def build_model_np(self):
        # Initialize all the parameters (Use Normal Distribution)
        # bu and bi are vectors (Note the dimension)
        self.bu = np.random.normal(scale = 1. / self.num_factors, size=[self.num_user])
        self.bi = np.random.normal(scale = 1. / self.num_factors, size=[self.num_item])

        # P and Q are matrices (Note the dimension)
        self.P = np.random.normal(scale=1. / self.num_factors, size=[self.num_user, self.num_factors])
        self.Q = np.random.normal(scale=1. / self.num_factors, size=[self.num_factors, self.num_item])
        print("Parameter Initialization Completed.")

    # Training using SGD algorithm
    def train_one_epoch_np(self):
        for uid, iid, ratings in list(zip(self.train_uid, self.train_iid, self.train_ratings)):
            # The estimated rating
            pred_r = self.mu + self.bu[uid] + self.bi[iid] + np.dot(self.Q[:, iid], self.P[uid, :])

            # Calculate the loss of this specific user-item pair
            error = ratings - pred_r

            # Update the parameters
            self.bu[uid] = self.bu[uid] + self.lr * (error - self.regs * self.bu[uid])
            self.bi[iid] = self.bi[iid] + self.lr * (error - self.regs * self.bi[iid])
            self.P[uid, :] = self.P[uid, :] + self.lr * (error * self.Q[:, iid] - self.regs * self.P[uid, :])
            self.Q[:, iid] = self.Q[:, iid] + self.lr * (error * self.P[uid, :] - self.regs * self.Q[:, iid])

    # Eval on the testing dataset
    def eval_one_epoch_np(self, epoch):
        rms_test_list, rms_train_list = [], []

        for uid, iid, ratings in list(zip(self.test_uid, self.test_iid, self.test_ratings)):
            rms_test_list.append(
                (self.mu + self.bu[uid] + self.bi[iid] + np.dot(self.Q[:, iid], self.P[uid, :]) - ratings) ** 2)

        for uid, iid, ratings in list(zip(self.train_uid, self.train_iid, self.train_ratings)):
            rms_train_list.append(
                (self.mu + self.bu[uid] + self.bi[iid] + np.dot(self.Q[:, iid], self.P[uid, :]) - ratings) ** 2)

        rms_test = np.sqrt(np.mean(rms_test_list))
        rms_train = np.sqrt(np.mean(rms_train_list))
        print("Epoch {0} Training: [RMS] {1} and Testing: [RMS] {2}".format(epoch, rms_train, rms_test))

    # Final training using the SGD algorithm
    def train_np(self):
        # Before Training
        self.eval_one_epoch_np(-1)
        # Start Training
        for i in range(self.epochs):
            self.train_one_epoch_np()
            self.eval_one_epoch_np(i)

### The Main Program


In [6]:
if __name__ == "__main__":
    model = MF( num_factors = 700,
                regularizer = 1e-3,
                learning_rate=0.005,
                epochs=50,
                batch_size=128)

    model.prepare_data()
    model.build_model_np()
    model.train_np()


Data Preview:
   uid  iid  ratings       time
0  196  242        3  881250949
1  186  302        3  891717742
2   22  377        1  878887116
3  244   51        2  880606923
4  166  346        1  886397596
Number of Users: 943
Number of Items: 1682
Sample Data: [[5 3 4 ... 0 0 0]]
Data Preparation Completed.
Parameter Initialization Completed.
Epoch -1 Training: [RMS] 1.12554580886021 and Testing: [RMS] 1.126324386619359
Epoch 0 Training: [RMS] 0.9971983841970238 and Testing: [RMS] 1.006368217098477
Epoch 1 Training: [RMS] 0.9663916476486082 and Testing: [RMS] 0.9797529540842551
Epoch 2 Training: [RMS] 0.9518714830249156 and Testing: [RMS] 0.967693601363532
Epoch 3 Training: [RMS] 0.9430844564517317 and Testing: [RMS] 0.9606561194749418
Epoch 4 Training: [RMS] 0.9371057214543206 and Testing: [RMS] 0.9560408384860962
Epoch 5 Training: [RMS] 0.9327672261413358 and Testing: [RMS] 0.9528179722846523
Epoch 6 Training: [RMS] 0.9294867549104853 and Testing: [RMS] 0.9504780957556713
Epoch 7 Tr

### We got better results than the SURPRISE library

Ours: RMSE: 0.919

SURPRISE: RMSE: 0.935


### Overfitting

The training RMSE keeps going down, while the testing RMSE goes down first and then goes up.

## Quiz Question 1: 

In this tutorial, we implement the SVD algorithm for matrix factorization. There are many other more advanced algorithms for matrix factorization.

Please use the [SURPRISE Library](http://surpriselib.com/) to see the results of SVD++ matrix factorization algorithm.


In [15]:
from surprise.model_selection import cross_validate
# Now import the SVD++ algorithm
from surprise import SVDpp

import warnings; warnings.simplefilter('ignore')

# Load the data
data, n_users, n_items = spr_loadData('ml-100k.data')

# Select Algorithm SVDpp and Run It
# Put your code here
# (Hint: follow the first piece of code using SURPRISE library, only two lines of code are needed here.)
algo = SVDpp()
cross_validate(algo, data, measures=['RMSE'], cv=5, verbose=True)
#cross_validate(SVDpp(), data, cv=5, measures=['RMSE','MAE'], verbose=False)
#mae include it & runs for a long time 

Evaluating RMSE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9161  0.9146  0.9135  0.9289  0.9228  0.9192  0.0058  
Fit time          229.84  232.02  227.35  229.80  228.62  229.53  1.55    
Test time         4.04    4.25    4.00    4.28    3.99    4.11    0.13    


{'fit_time': (229.83723187446594,
  232.02133202552795,
  227.35451984405518,
  229.80094861984253,
  228.6177065372467),
 'test_rmse': array([0.91606047, 0.91458999, 0.9134821 , 0.92888334, 0.92279902]),
 'test_time': (4.042974233627319,
  4.254604816436768,
  4.0020911693573,
  4.276906728744507,
  3.9913265705108643)}

## Quiz Question 2:

In our introduction to recommender systems, we have already covered the following methods: random, user-based collaborative filtering, item-based collaborative filtering, content-based recommendation, and matrix factorization using SVD. So, which one is better?

Please complete the following code to compare these methods (we only use the SURPRISE library here)

In [14]:
from surprise.model_selection import cross_validate
from surprise import NormalPredictor,KNNBasic,KNNWithMeans,SVD

import warnings; warnings.simplefilter('ignore')

# Load the data (you don't need to change this)
data, n_users, n_items = spr_loadData('ml-100k.data')

# Choose the algorithms (you don't need to change this part)
sim_options_userCF = {'name': 'cosine',
                       'user_based': True  # compute similarities between users
                      }
sim_options_itemCF = {'name': 'cosine',
                       'user_based': False  # compute similarities between users
                      }

algo1 = NormalPredictor()
algo2 = KNNBasic(k=n_users,sim_options=sim_options_userCF)
algo3 = KNNBasic(k=n_items,sim_options=sim_options_itemCF)
algo4 = KNNWithMeans(k=n_users,sim_options=sim_options_userCF)
algo5 = KNNWithMeans(k=n_items,sim_options=sim_options_itemCF)
algo6 = SVD()

# Run the algorithms
cross_validate(algo1, data, measures=['RMSE'], cv=5, verbose=True) # You don't need to change this line
# Now run the other algorithms
# Put your code here (Hint: follow the above line of code to run algo2 to algo6. 
#You only need to write 5 lines of code. Copy and Paste with little modification)
cross_validate(algo2, data, measures=['RMSE'], cv=5, verbose=True) # You don't need to change this line
cross_validate(algo3, data, measures=['RMSE'], cv=5, verbose=True) # You don't need to change this line
cross_validate(algo4, data, measures=['RMSE'], cv=5, verbose=True) # You don't need to change this line
cross_validate(algo5, data, measures=['RMSE'], cv=5, verbose=True) # You don't need to change this line
cross_validate(algo6, data, measures=['RMSE'], cv=5, verbose=True) # You don't need to change this line

Evaluating RMSE of algorithm NormalPredictor on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.5186  1.5088  1.5180  1.5251  1.5183  1.5178  0.0052  
Fit time          0.14    0.17    0.17    0.17    0.17    0.17    0.01    
Test time         0.14    0.14    0.14    0.14    0.14    0.14    0.00    
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0193  1.0298  1.0215  1.0252  1.0248  1.0241  0.0036  
Fit time          1.28    1.30    1.26    1.27    1.27 

{'fit_time': (5.965492486953735,
  6.01469612121582,
  5.97943377494812,
  5.9966583251953125,
  5.975028991699219),
 'test_rmse': array([0.93240281, 0.93564391, 0.94305857, 0.94330331, 0.92402091]),
 'test_time': (0.1633281707763672,
  0.34355926513671875,
  0.16055798530578613,
  0.3498365879058838,
  0.16109061241149902)}